In [1]:
from Bio.Seq import Seq
import pandas as pd

In [2]:
seq_2_screen_file = 'seq_2_screen/example_seq_2_screen.csv'  # path/name of csv containning sequences in which to look for PAMs
PAM_NGG = ['AGG', 'TGG', 'CGG', 'GGG', ] # PAMs to look for
seq_2_screen = pd.read_csv(seq_2_screen_file)

In [3]:
output_file = 'gRNAs_2_screen/example_gRNA_2_screen.csv' # path/name of csv output file with gRNAs to be screened with Wayfinder
all_but_spacer_gRNA = 'GTTTTAGAGCTAGAAATAGCAAGTTAAAATAAGGCTAGTCCGTTATCAACTTGAAAAAGTGGCACCGAGTCGGTGCTTTTTTT' # change between sgRNA or scRNA
Spacer_Length = 20

In [4]:
def find_PAM_bottom(ORF, Locus_tag, PAM = ['AGG', 'TGG', 'CGG', 'GGG']):

    """Receive ORF as Seq and PAM as a set of compatible PAMs"""
    
    if type(ORF) == str:   #convert string to Seq
        ORF = Seq(ORF)
    else:
        pass
    
    ORF = ORF.upper()
    
    PAM_rev = []

    PAM_pos = []
    PAM_seq = []
    PAM_strand = []
    PAM_3nt = []
    Name = []

    for pam in PAM:
        PAM_rev.append(str(Seq(pam).reverse_complement()))

    for i in range(len(ORF)-20):
        if ORF[i:i+3] in PAM_rev:
            position = i+3
            spacer = str(ORF[i+3:i+23].reverse_complement())
            #print(position, spacer)

            PAM_pos.append(position)
            PAM_seq.append(spacer)
            PAM_strand.append(-1)
            PAM_3nt.append(str(ORF[i:i+3].reverse_complement()))
            Name.append(Locus_tag)
            
#        elif ORF[i:i+3] in PAM_rev:
#            position = i+3
#            spacer = str(ORF[i+3:i+3+20].reverse_complement())
#            #print(position, spacer)

#            PAM_pos.append(position)
#            PAM_seq.append(spacer)
#            PAM_strand.append(1)

    df_PAM = pd.DataFrame({'Locus_tag': Name, 'Position': PAM_pos, 'Spacer': PAM_seq, 'PAM': PAM_3nt, 'Strand': PAM_strand})
    
    return(df_PAM)

In [5]:
# looking for spacers -- CK
output = {}
for index, row in seq_2_screen.iterrows():
    output[row['Locus_tag']] = find_PAM_bottom(row['ORF'], row['Locus_tag'], PAM = PAM_NGG)

df_target = pd.DataFrame({'Locus_tag': [], 'Position': [], 'Spacer': [], 'PAM': [], 'Strand': []})
for gene in output:
    df_target_temp = output[gene]
    df_target = df_target.append(df_target_temp)

In [6]:
# making sure to put the spacers in the same format as Wayfinder -- RC

input_4_wayfinder = pd.DataFrame(columns=['Unique_Name', 'Spacer_Length', 'Spacer_Position','Raw_Fluorescence_1','Raw_Fluorescence_2','Raw_Fluorescence_3','Avg_Fluorescence','Date','Sequence','upstream'])

c = 1
for target in df_target.iterrows():
    row = {'Unique_Name':target[1]['Locus_tag']+'__'+str(target[1]['Position']), 'Spacer_Length':Spacer_Length,'Spacer_Position':6,'Sequence':target[1]['Spacer']+all_but_spacer_gRNA, 'upstream':0}
    input_4_wayfinder = input_4_wayfinder.append(row, ignore_index = True)
    c+=1

input_4_wayfinder.to_csv(output_file, index = False)